In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def prophet_forecast(
    series,
    district_name,
    forecast_periods=12,
    yearly_seasonality='auto',
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    holidays=None,
    plot_forecast=True
):
    """
    Prophet time series forecasting with error handling and RMSE calculation.
    Returns (metrics, forecast_df) or (None, None) on error.
    """
    try:
        # 0. Directory setup
        os.makedirs('Prophet_Forecasts', exist_ok=True)

        # 1. Prepare DataFrame for Prophet
        df = pd.DataFrame({'ds': series.index, 'y': series.values})
        df = df.dropna()
        if len(df) < forecast_periods + 12:
            raise ValueError(f"Series too short ({len(df)} points) for {forecast_periods}-step forecast.")

        # 2. Train/test split
        train = df.iloc[:-forecast_periods]
        test = df.iloc[-forecast_periods:]

        # 3. Model setup
        model = Prophet(
            yearly_seasonality=yearly_seasonality,
            weekly_seasonality=weekly_seasonality,
            daily_seasonality=daily_seasonality,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            holidays=holidays
        )
        model.fit(train)

        # 4. Forecast
        future = model.make_future_dataframe(periods=forecast_periods, freq='MS')
        forecast = model.predict(future)

        # 5. Extract forecasted values for test period
        forecast_test = forecast.set_index('ds').loc[test['ds']]
        y_pred = forecast_test['yhat'].values
        y_test = test['y'].values

        # 6. Metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        metrics = {
            'district': district_name,
            'rmse': rmse,
            'forecast_periods': forecast_periods
        }

        # 7. Save forecast DataFrame
        forecast_df = pd.DataFrame({
            'district': district_name,
            'date': test['ds'].values,
            'actual': y_test,
            'forecast': y_pred
        })
        forecast_df.to_csv(
            os.path.join('Prophet_Forecasts', f'{district_name}_forecast.csv'),
            index=False
        )

        # 8. Plotting
        if plot_forecast:
            plt.figure(figsize=(12, 6))
            plt.plot(df['ds'], df['y'], label='Original')
            plt.plot(forecast_df['date'], forecast_df['forecast'], label='Prophet Forecast', color='crimson')
            plt.scatter(forecast_df['date'], forecast_df['actual'], label='Actual (Test)', color='black')
            plt.title(f'{district_name} Prophet Forecast (RMSE: {rmse:.2f})')
            plt.legend()
            plt.savefig(os.path.join('Prophet_Forecasts', f'{district_name}_forecast_plot.png'))
            plt.close()

        return metrics, forecast_df

    except Exception as e:
        print(f"⚠️ Skipping {district_name}: {str(e)}")
        return None, None

In [4]:
districts = data['district'].unique()
rmse_values = []

# Usage Example


def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I55"].asfreq('MS')
        
        metrics, forecast = prophet_forecast(
            series=ts,
            district_name=district,
            forecast_periods=6,  # Forecast the last year
            yearly_seasonality='auto',
            weekly_seasonality=False,
            daily_seasonality=False
        )

        
    
        # Show results
        print("=== Metrics ===")
        if metrics:
            print(metrics)
        print("\n=== Forecast Data ===")
        print(forecast)
    
    return results
run_for_each_district()

16:00:43 - cmdstanpy - INFO - Chain [1] start processing
16:00:44 - cmdstanpy - INFO - Chain [1] done processing
16:00:44 - cmdstanpy - INFO - Chain [1] start processing


=== Metrics ===
{'district': 'AHMEDNAGAR', 'rmse': 2307.7065807141907, 'forecast_periods': 6}

=== Forecast Data ===
     district       date  actual     forecast
0  AHMEDNAGAR 2020-10-01    6799  5849.173853
1  AHMEDNAGAR 2020-11-01    5248  5782.043775
2  AHMEDNAGAR 2020-12-01    4923  5542.589475
3  AHMEDNAGAR 2021-01-01    4542  9218.391484
4  AHMEDNAGAR 2021-02-01    4301  6484.628788
5  AHMEDNAGAR 2021-03-01    3941  5876.180198


16:00:44 - cmdstanpy - INFO - Chain [1] done processing
16:00:44 - cmdstanpy - INFO - Chain [1] start processing


=== Metrics ===
{'district': 'AKOLA', 'rmse': 1134.5717133680312, 'forecast_periods': 6}

=== Forecast Data ===
  district       date  actual     forecast
0    AKOLA 2020-10-01    2342  3473.345130
1    AKOLA 2020-11-01    2550  2534.169979
2    AKOLA 2020-12-01    2513  1952.443898
3    AKOLA 2021-01-01    2215  2928.316723
4    AKOLA 2021-02-01    1967  1083.963959
5    AKOLA 2021-03-01    2265  4465.120202


16:00:44 - cmdstanpy - INFO - Chain [1] done processing
16:00:45 - cmdstanpy - INFO - Chain [1] start processing


...

16:00:58 - cmdstanpy - INFO - Chain [1] done processing
16:00:59 - cmdstanpy - INFO - Chain [1] start processing


=== Metrics ===
{'district': 'WASHIM', 'rmse': 223.77965709169757, 'forecast_periods': 6}

=== Forecast Data ===
  district       date  actual    forecast
0   WASHIM 2020-10-01     696  752.543315
1   WASHIM 2020-11-01     627  593.676085
2   WASHIM 2020-12-01     616  803.549319
3   WASHIM 2021-01-01     509  720.131293
4   WASHIM 2021-02-01     477  861.270341
5   WASHIM 2021-03-01     521  783.186030


16:00:59 - cmdstanpy - INFO - Chain [1] done processing


=== Metrics ===
{'district': 'YAVATMAL', 'rmse': 1251.0779732487774, 'forecast_periods': 6}

=== Forecast Data ===
   district       date  actual     forecast
0  YAVATMAL 2020-10-01    3708  4538.350681
1  YAVATMAL 2020-11-01    3219  4007.932485
2  YAVATMAL 2020-12-01    3613  1762.432797
3  YAVATMAL 2021-01-01    3199  3752.219947
4  YAVATMAL 2021-02-01    2808  4889.926163
5  YAVATMAL 2021-03-01    3435  3554.214173
⚠️ Skipping MUMBAI SUBURBAN: Series too short (12 points) for 6-step forecast.
=== Metrics ===

=== Forecast Data ===
None


{}

In [5]:
districts

array(['AHMEDNAGAR', 'AKOLA', 'AMRAVATI', 'AURANGABAD', 'BEED',
       'BHANDARA', 'BULDHANA', 'CHANDRAPUR', 'DHULE', 'GADCHIROLI',
       'GONDIA', 'HINGOLI', 'JALGAON', 'JALNA', 'KOLHAPUR', 'LATUR',
       'MUMBAI', 'NAGPUR', 'NANDED', 'NANDURBAR', 'NASHIK', 'OSMANABAD',
       'PALGHAR', 'PARBHANI', 'PUNE', 'RAIGAD', 'RATNAGIRI', 'SANGLI',
       'SATARA', 'SINDHUDURG', 'SOLAPUR', 'THANE', 'WARDHA', 'WASHIM',
       'YAVATMAL', 'MUMBAI SUBURBAN'], dtype=object)